# [1484. Group Sold Products By The Date](https://leetcode.com/problems/group-sold-products-by-the-date/description/?envType=study-plan-v2&envId=top-sql-50)

Table Activities:

<pre>+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| sell_date   | date    |
| product     | varchar |
+-------------+---------+</pre>
There is no primary key (column with unique values) for this table. It may contain duplicates.
Each row of this table contains the product name and the date it was sold in a market.
 

Write a solution to find for each date the number of different products sold and their names.

The sold products names for each date should be sorted lexicographically.

Return the result table ordered by sell_date.

The result format is in the following example.

 

Example 1:

Input: 
Activities table:
<pre>+------------+------------+
| sell_date  | product     |
+------------+------------+
| 2020-05-30 | Headphone  |
| 2020-06-01 | Pencil     |
| 2020-06-02 | Mask       |
| 2020-05-30 | Basketball |
| 2020-06-01 | Bible      |
| 2020-06-02 | Mask       |
| 2020-05-30 | T-Shirt    |
+------------+------------+</pre>
Output: 
<pre>+------------+----------+------------------------------+
| sell_date  | num_sold | products                     |
+------------+----------+------------------------------+
| 2020-05-30 | 3        | Basketball,Headphone,T-shirt |
| 2020-06-01 | 2        | Bible,Pencil                 |
| 2020-06-02 | 1        | Mask                         |
+------------+----------+------------------------------+</pre>
Explanation: 
For 2020-05-30, Sold items were (Headphone, Basketball, T-shirt), we sort them lexicographically and separate them by a comma.
For 2020-06-01, Sold items were (Pencil, Bible), we sort them lexicographically and separate them by a comma.
For 2020-06-02, the Sold item is (Mask), we just return it.

In [1]:
# pandas schema

import pandas as pd

data = [['2020-05-30', 'Headphone'], ['2020-06-01', 'Pencil'], ['2020-06-02', 'Mask'], ['2020-05-30', 'Basketball'],
        ['2020-06-01', 'Bible'], ['2020-06-02', 'Mask'], ['2020-05-30', 'T-Shirt']]
activities = pd.DataFrame(data, columns=['sell_date', 'product']).astype(
    {'sell_date': 'datetime64[ns]', 'product': 'object'})

In [2]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

#pyspark schema

spark = SparkSession.builder.getOrCreate()

activities_df = spark.createDataFrame(activities)
activities_df.show(truncate=False)

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/11 21:27:08 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/11 21:27:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/11 21:27:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-------------------+----------+
|sell_date          |product   |
+-------------------+----------+
|2020-05-30 00:00:00|Headphone |
|2020-06-01 00:00:00|Pencil    |
|2020-06-02 00:00:00|Mask      |
|2020-05-30 00:00:00|Basketball|
|2020-06-01 00:00:00|Bible     |
|2020-06-02 00:00:00|Mask      |
|2020-05-30 00:00:00|T-Shirt   |
+-------------------+----------+


In [3]:
# solving in spark dataframe

activities_df \
    .groupBy('sell_date') \
    .agg(
    F.count_distinct('product').alias('num_sold'),
    F.sort_array(F.collect_set(F.col('product'))).alias('products')
    ) \
    .orderBy('sell_date') \
    .show(truncate=False)

#to get rid of those brackets, we can use F.concat_ws(',',F.col()) but its type will be converted to string.

+-------------------+--------+--------------------------------+
|sell_date          |num_sold|products                        |
+-------------------+--------+--------------------------------+
|2020-05-30 00:00:00|3       |[Basketball, Headphone, T-Shirt]|
|2020-06-01 00:00:00|2       |[Bible, Pencil]                 |
|2020-06-02 00:00:00|1       |[Mask]                          |
+-------------------+--------+--------------------------------+


In [4]:
# solving in spark SQL

activities_df.createOrReplaceTempView('activities')

spark.sql('''
select 
    sell_date,
    count(distinct product) as num_sold,
    sort_array(collect_set(product)) as products
from activities
group by sell_date
order by sell_date;
''').show(truncate=False)

+-------------------+--------+--------------------------------+
|sell_date          |num_sold|products                        |
+-------------------+--------+--------------------------------+
|2020-05-30 00:00:00|3       |[Basketball, Headphone, T-Shirt]|
|2020-06-01 00:00:00|2       |[Bible, Pencil]                 |
|2020-06-02 00:00:00|1       |[Mask]                          |
+-------------------+--------+--------------------------------+


In [5]:
### MySQL equivalent:

'''
select 
    sell_date,
    count(distinct product) as num_sold,
    group_concat(distinct product order by product asc) as products
from activities
group by sell_date
order by sell_date;
'''

'\nselect \n    sell_date,\n    count(distinct product) as num_sold,\n    group_concat(distinct product order by product asc) as products\nfrom activities\ngroup by sell_date\norder by sell_date;\n'

In [6]:
spark.stop()